**Формулировка задачи**

Пусть $\mathbb{F}_p$ обозначает конечное поле порядка $p$, где $p$ — простое число. Построим форму (т.е. однородный многочлен) четвертой степени от трех переменных $x_1, x_2, x_3$, которая является анизотропной (т.е. имеет в качестве корня лишь $x_1 = 0, x_2 = 0, x_3 = 0$) над полем $\mathbb{F}_p$, но изотропной (т.е. имеет некоторый корень отличный от нулевого) над его кубическим расширением, порожденным некоторым неприводимым многочленом третьей степени с коэффициентами из $\mathbb{F}_p$.


**Решение**

In [51]:
import sympy as sp
import math
from itertools import product

In [52]:
prime = 5

**Функция проверки формы на анизотропность**

Будем проверять форму $q(x_1, x_2, x_3) = d_1x_1^4 + d_2x_2^4+ d_3x_3^4$, где $d_1, d_2, d_3 \in \mathbb{F}_p$ на анизотропность с помощью перебора всех возможных элементов из $\mathbb{F}_p$.

In [55]:
def check_for_anisotropicity(coeffs: tuple[int]) -> bool:
    for x_1 in range(prime):
        for x_2 in range(prime):
            for x_3 in range(prime):
                if (coeffs[0]*(x_1**4)+coeffs[1]*(x_2**4)+coeffs[2]*(x_3**4))%prime==0 and (x_1,x_2,x_3)!=(0,0,0):
                    return False
    return True

**Функция проверки многочлена на неприводимость**

Будем проверять многочлен $f(x) = x^3 + f_1x + f_2$, где $f_1, f_2 \in \mathbb{F}_p$ на неприводимость. Так как это многочлен третьей степени, то он может разложиться лишь на множители вида $x-x_0$ и $x^2+cx+d$. Поэтому для неприводимости $f(x)$ достаточно отсутствия у него корней. Так как $f\in\mathbb{F}_p[x]$, то искать его корни можно методом перебора.

In [58]:
def irred_cubic_polynom(coefs: tuple[int]) -> bool:
    for x in range(prime):
        if (x**3+coefs[0]*x+coefs[1])%prime==0:
            return False
    return True

**Поиск анизотропной формы над $\mathbb{F}_p$**

In [60]:
def possible_forms() -> list[tuple[int]]:
    form_coeffs = []
    for coeff1, coeff2, coeff3 in product(range(1, prime), repeat=3):
        if check_for_anisotropicity(cfs:=(coeff1, coeff2, coeff3)):
            form_coeffs.append(cfs)
    return form_coeffs

**Поиск неприводимого многочлена с коэффициентами из $\mathbb{F}_p$**

In [62]:
def possible_poly() -> list[tuple[int]]:
    poly_coeffs = []
    for coeff1, coeff2 in product(range(prime), repeat=2):
        if irred_cubic_polynom(cfs:=(coeff1, coeff2)):
            poly_coeffs.append(cfs)
    return poly_coeffs

**Получение уравнений на потенциальный корень из кубического расширения**

В найденную анизотропную над $\mathbb{F}_p$ форму $q(x_1, x_2, x_3) = d_1x_1^4 + d_2x_2^4+ d_3x_3^4$, подставим произвольные элементы $y_1, y_2, y_3\in \mathbb{F}_p(\alpha)$, где $\alpha^3 + f_1\alpha + f_2 = 0$ (то есть $\alpha$ - корень неприводимого над $\mathbb{F}_p$ многочлена $f(t) = t^3 + f_1t + f_2$). Произвольный элемент $y_i$ из $\mathbb{F}_p(\alpha)$ имеет вид $y_i = a_it^2 + b_it + c_i$, где $a_i, b_i, c_i \in \mathbb{F}_p$. Подставив $y_1, y_2, y_3$ в $q(x_1, x_2, x_3)$ собёрем вместе слагаемые при одинаковых степенях $t$, после чего приравняем их к нулю. Таким образом получим систему уравнений на потенциальный корень.  

In [65]:
def equations_for_counterex(form_coeffs, poly_coeffs) -> list[str]:
    t = sp.symbols('t')
    a1, a2, a3 = sp.symbols('a1 a2 a3')
    b1, b2, b3 = sp.symbols('b1 b2 b3')
    c1, c2, c3 = sp.symbols('c1 c2 c3')
    x1, x2, x3 = sp.symbols('x1 x2 x3')
    
    d1, d2, d3 = form_coeffs
    f1, f2 = poly_coeffs

    form = sp.Poly(d1*x1**4+d2*x2**4+d3*x3**4, x1, x2, x3)

    new_x1 = a1*t**2+b1*t+c1
    new_x2 = a2*t**2+b2*t+c2
    new_x3 = a3*t**2+b3*t+c3
    
    form1 = form.subs({x1:new_x1,x2:new_x2,x3:new_x3})
    form1 = sp.Poly(form1,t).rem(sp.Poly(t**3+f1*t+f2,t))

    return list(map(str,form1.all_coeffs()))

**Решение найденных уравнений**

Решим систему, полученную на предыдущем шаге с помощью перебора.

In [68]:
def solving_equations(equations) -> str | None:
    eq1, eq2, eq3 = equations[0], equations[1], equations[2]
    for a1, a2, a3, b1, b2, b3, c1, c2, c3 in product(range(prime), repeat=9):
       if any(var != 0 for var in (a1, a2, a3, b1, b2, b3, c1, c2, c3)):
           values = {
               'a1': a1,
               'a2': a2,
               'a3': a3,
               'b1': b1,
               'b2': b2,
               'b3': b3,
               'c1': c1,
               'c2': c2,
               'c3': c3
           }
           if eval(eq1, {}, values)%prime==0 and eval(eq2, {}, values)%prime==0 and eval(eq3, {}, values)%prime==0:
               return f'ROOT {values}'

**Получение корня, если он существует**

Получим анизотропную форму, неприводимый многочлен и корень формы из кубического расширения, если это возможно.

In [71]:
def get_counterexample() -> tuple[str] | str:
    for form in possible_forms():
        for poly in possible_poly():
            if (ans:=solving_equations(equations_for_counterex(form, poly))):
                return f'FORM: {form}', f'POLY: {poly}', ans
    return 'NO COUNTEREXAMPLES'

In [72]:
get_counterexample()

('FORM: (1, 1, 1)',
 'POLY: (1, 1)',
 "ROOT {'a1': 0, 'a2': 0, 'a3': 1, 'b1': 0, 'b2': 1, 'b3': 2, 'c1': 1, 'c2': 0, 'c3': 1}")